In [1]:
import os

os.chdir("../")

In [2]:
%pwd

'/Users/vince/code/projects/aws-mlflow-mlops'

In [3]:
# Update entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
# Update configuration manager
from src.aws_mlflow_mlops.constants import *
from src.aws_mlflow_mlops.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    """
    The ConfigurationManager class is responsible for managing the configuration of the data ingestion process.
    It reads configuration, parameters, and schema from YAML files, and creates necessary directories.

    Attributes:
        config (dict): Configuration read from a YAML file.
        params (dict): Parameters read from a YAML file.
        schema (dict): Schema read from a YAML file.
    """

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        """
        The constructor for ConfigurationManager class.

        Parameters:
            config_filepath (str): The path to the configuration YAML file.
            params_filepath (str): The path to the parameters YAML file.
            schema_filepath (str): The path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        The method to get the data transformation configuration.

        This method retrieves the data transformation configuration from the main configuration object,
        creates a DataTransformationConfig instance with the root directory and data path from the configuration,
        and returns this instance.

        Returns:
            DataTransformationConfig: An instance of DataTransformationConfig with the root directory and data path from the configuration.
        """
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [13]:
#Update components
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from src.aws_mlflow_mlops import logger


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into train and test sets.")
        logger.info(f"Train shape: {train.shape}\n Test shape:{test.shape}")

        print(f"Train shape: {train.shape}\n Test shape:{test.shape}")

In [14]:
try:
    config = ConfigurationManager()
    data_transformaton_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformaton_config)
    data_transformation.train_test_splitting()
except Exception as e:
    logger.exception("Failed to run data transformation.")
    raise e

[2023-11-14 15:58:09,600: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-14 15:58:09,602: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-14 15:58:09,604: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-14 15:58:09,605: INFO: common: created directory at: artifacts]
[2023-11-14 15:58:09,606: INFO: common: created directory at: artifacts/data_transformation]
[2023-11-14 15:58:09,668: INFO: 2936060203: Split data into train and test sets.]
[2023-11-14 15:58:09,669: INFO: 2936060203: Train shape: (1279, 12)
 Test shape:(320, 12)]
Train shape: (1279, 12)
 Test shape:(320, 12)
